In [7]:
import pyodbc
from dotenv import load_dotenv
import os

# Charger les variables d'environnement
load_dotenv()  # Assurez-vous que le chemin vers .env est correct si nécessaire

# Utiliser les variables d'environnement
server = os.getenv('DB_SERVER')
database = os.getenv('DB_NAME')
username = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
driver = os.getenv('DB_DRIVER')

print(server)

sqldatabase-lemay.database.windows.net


In [8]:
# Définir la chaîne de connexion
conn_str = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"Encrypt=yes;"  # Nécessaire pour Azure
    f"TrustServerCertificate=no;"  # Nécessaire pour Azure
    f"Connection Timeout=30;"
)


In [9]:
# Nom du schéma
schema_name = 'appmovieschema'  # Remplacez par le nom de votre schéma

# Établir la connexion
try:
    with pyodbc.connect(conn_str) as conn:
        print("Connexion réussie.")
        cursor = conn.cursor()

        # Créer les tables selon le schéma fourni
        cursor.execute(f"IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = N'{schema_name}') EXEC('CREATE SCHEMA {schema_name}')")
        cursor.execute(f'''
        CREATE TABLE {schema_name}.User_Table (
            user_id INT PRIMARY KEY,
            pseudo NVARCHAR(255),
            email NVARCHAR(255),
            password NVARCHAR(255)
        )
        ''')
        cursor.execute(f'''
        CREATE TABLE {schema_name}.Movie_Table (
            movie_id INT PRIMARY KEY,
            title NVARCHAR(255),
            release_date DATE,
            poster_link NVARCHAR(255)
        )
        ''')
        cursor.execute(f'''
        CREATE TABLE {schema_name}.Genre_Table (
            genre_id INT PRIMARY KEY,
            genre_name NVARCHAR(255)
        )
        ''')
        cursor.execute(f'''
        CREATE TABLE {schema_name}.UserMovieList_Table (
            user_id INT,
            movie_id INT,
            FOREIGN KEY (user_id) REFERENCES {schema_name}.User_Table(user_id),
            FOREIGN KEY (movie_id) REFERENCES {schema_name}.Movie_Table(movie_id)
        )
        ''')
        cursor.execute(f'''
        CREATE TABLE {schema_name}.MovieGenre_Table (
            movie_id INT,
            genre_id INT,
            FOREIGN KEY (movie_id) REFERENCES {schema_name}.Movie_Table(movie_id),
            FOREIGN KEY (genre_id) REFERENCES {schema_name}.Genre_Table(genre_id)
        )
        ''')

        # Valider la création
        conn.commit()
        print("Les tables ont été créées avec succès dans le schéma", schema_name)

except pyodbc.Error as e:
    print("Erreur lors de la connexion à la base de données Azure SQL:", e)

Connexion réussie.
Les tables ont été créées avec succès dans le schéma appmovieschema
